# VacationPy
----

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
from matplotlib import pyplot as plt

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_py = pd.read_csv("Weather Output Data/cities_output.csv")
weather_py.head(10)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Flinders,0,AU,1628633200,66,-34.5833,150.8552,60.06,9.48
1,Ranong,100,TH,1628633343,88,9.9658,98.6348,79.20,10.25
2,Mataura,27,NZ,1628633167,94,-46.1927,168.8643,46.47,3.40
3,Hong Kong,100,HK,1628633243,85,22.2855,114.1577,85.32,1.99
4,Barrow,90,US,1628633343,83,71.2906,-156.7887,37.42,9.22
5,Wilmington,59,US,1628632719,73,34.2257,-77.9447,88.95,3.00
6,Katherine,63,AU,1628633344,73,-14.4667,132.2667,68.32,3.44
7,Belize,5,BZ,1628633344,44,17.2500,-88.7500,89.92,7.20
8,Kaitangata,16,NZ,1628633344,84,-46.2817,169.8464,46.20,6.89
9,Achit,0,RU,1628633345,89,56.7985,57.8994,48.61,4.43


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
locations = weather_py[["Lat", "Lng"]]
humidity = weather_py["Humidity"]

In [5]:
#Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Display Conditions with specific values
city_conditions = weather_py.loc[(weather_py["Wind Speed"] <= 10) & (weather_py["Cloudiness"] == 0) & \
                                   (weather_py["Max Temp"] >= 70) & (weather_py["Max Temp"] <= 80)].dropna()

city_conditions

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
144,Changping,0,CN,1628633388,60,40.2186,116.2197,71.24,7.63
214,Vila Velha,0,BR,1628633361,83,-20.3297,-40.2925,71.55,6.91
254,Yangi Marg`ilon,0,UZ,1628633424,46,40.4272,71.7189,74.12,6.91
259,Gazojak,0,TM,1628633426,28,41.1875,61.4036,79.34,9.71
267,Hyères,0,FR,1628633429,61,43.1167,6.1167,76.68,2.30
400,Kaka,0,TM,1628633472,19,37.3482,59.6143,77.92,1.74
473,Malanje,0,AO,1628633494,23,-9.5402,16.3410,70.11,1.79
504,Şile,0,TR,1628633504,67,41.1754,29.6133,75.87,1.01
531,Abu Samrah,0,SY,1628633514,62,35.3029,37.1841,77.74,7.11
535,Arys,0,KZ,1628633515,35,42.4252,68.7950,75.42,9.66


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel_df
hotel_df = city_conditions.loc[:,["City","Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Lat,Lng,Hotel Name
144,Changping,CN,40.2186,116.2197,
214,Vila Velha,BR,-20.3297,-40.2925,
254,Yangi Marg`ilon,UZ,40.4272,71.7189,
259,Gazojak,TM,41.1875,61.4036,
267,Hyères,FR,43.1167,6.1167,
400,Kaka,TM,37.3482,59.6143,
473,Malanje,AO,-9.5402,16.3410,
504,Şile,TR,41.1754,29.6133,
531,Abu Samrah,SY,35.3029,37.1841,
535,Arys,KZ,42.4252,68.7950,


In [8]:
#Create a starter URL to build on with Parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to parameteres
    params["location"] = f"{lat},{lng}"

    # assemble url with Parameters
    print(f"Retrieving Room Keys for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # Get Results
    results = response['results']

    # Create pull
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, create an exception and tell them to hit the road
    except (KeyError, IndexError):
        print("No Room in the Inn. Hit the Road, Jack.")
        
    print("------------")
    
    # Give it a second
    time.sleep(1)

# Let them know that their search is over
print("-------End of Search. Have a Nice Stay!-------")

Retrieving Room Keys for Index 144: Changping.
Closest hotel in Changping is Memory Hotel.
------------
Retrieving Room Keys for Index 214: Vila Velha.
Closest hotel in Vila Velha is Hotel Senac Ilha do Boi.
------------
Retrieving Room Keys for Index 254: Yangi Marg`ilon.
Closest hotel in Yangi Marg`ilon is TANTANA HOTEL FERGANA.
------------
Retrieving Room Keys for Index 259: Gazojak.
No Room in the Inn. Hit the Road, Jack.
------------
Retrieving Room Keys for Index 267: Hyères.
Closest hotel in Hyères is Logis Hôtel le Ceinturon.
------------
Retrieving Room Keys for Index 400: Kaka.
No Room in the Inn. Hit the Road, Jack.
------------
Retrieving Room Keys for Index 473: Malanje.
Closest hotel in Malanje is Palanca Hotel.
------------
Retrieving Room Keys for Index 504: Şile.
Closest hotel in Şile is Phellos Suites.
------------
Retrieving Room Keys for Index 531: Abu Samrah.
No Room in the Inn. Hit the Road, Jack.
------------
Retrieving Room Keys for Index 535: Arys.
Closest hot

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer content
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))